In [1]:
# Imports
import requests
import yaml

from openai import OpenAI

In [2]:
# Reading config
with open("/Users/fitter0happier/Desktop/Coding/NLP/21Mind/config.yaml", "r") as file:
    config = yaml.safe_load(file)
    address = config["Providers"]["DEEPSEEKLOCAL"]

In [3]:
# Defining API calls
test = "/v1/models"
completions = "/v1/chat/completions"

In [5]:
# Testing
x = requests.get(address + test)
x.content

b'{\n  "data": [\n    {\n      "id": "deepseek-r1-distill-qwen-7b",\n      "object": "model",\n      "owned_by": "organization_owner"\n    },\n    {\n      "id": "text-embedding-nomic-embed-text-v1.5",\n      "object": "model",\n      "owned_by": "organization_owner"\n    }\n  ],\n  "object": "list"\n}'

In [23]:
# DeepSeek R1 Qwen model
class DEEPSEEKR1QWEN:
    def __init__(self) -> None:
        with open("/Users/fitter0happier/Desktop/Coding/NLP/21Mind/config.yaml", "r") as file:
            config = yaml.safe_load(file)

        self.base_url = config["Providers"]["DEEPSEEKLOCAL"]
        self.model = config["IDs"]["DEEPSEEKR1QWEN"]
        self.completions = "/v1/chat/completions"

    def response(self, mes):
        headers = {
            "Content-Type": "application/json"
        }
        data = {
            "model": self.model,
            "messages": mes,
            "temperature": 0.7,
            "max_tokens": -1,
            "stream": False
        }

        response = requests.post(self.base_url+self.completions, headers=headers, json=data)
        return response

In [ ]:
# Test response
llm = DEEPSEEKR1QWEN()
message=[
    {
      "role": "user",
      "content": """
        You are playing BlackJack, the dealer has a 7 of Spades, 
        you have a Queen of Hearts and a Two of Diamonds, 
        availabe actions are Hit and Stand. 
        What is your action? Always write "<response>\n" after you are finished reasoning.
      """
    }
]

response = llm.response(message).json()
print(response['choices'][0]['message']['content'])